### T-Period Consumption-Saving Model with Labor Supply

Solves a $T$-period consumption-saving model with endogenous labor supply and no uncertainty.

At each period $ t = 1, \dots, T $, the agent chooses consumption  $c_t$ and labor supply $\ell_t \in [0, 1] $ to maximize lifetime utility, trading off consumption gains with labor disutility:

$$
\begin{align}
V_t(a_t) &= \max_{c_t > 0, \; \ell_t \in [0,1]} \left\{ \frac{c_t^{1-\rho}}{1-\rho} - \varphi \cdot \frac{\ell_t^{1+\eta}}{1+\eta} + \beta \cdot V_{t+1}(a_{t+1}) \right\} \\
\text{s.t.} \quad a_{t+1} &= (1 + r) \left[a_t + y_t + w_t \cdot \ell_t - c_t \right] \\
a_t &\geq 0 \quad \text{(No borrowing)}
\end{align}
$$


### Model Parameter Settings

| **Parameter**   | **Value**                             | **Description**                             |
|------------------|-----------------------------------------|---------------------------------------------|
| `T`              | 50                                      | Number of periods                           |
| `β` (`beta`)     | 0.98                                    | Discount factor                             |
| `ρ` (`rho`)      | 2.0                                     | Relative risk aversion (CRRA)               |
| `ϕ` (`ϕ`)        | 1.0                                     | Weight on disutility of labor               |
| `η` (`eta`)      | 1.5                                     | Inverse Frisch elasticity                   |
| `y`              | 30,000                                  | Constant non-labor income                   |
| `w`              | 20,000                                  | Constant wage rate                          |
| `r`              | 0.02                                    | Interest rate                               |
| `a_max`          | 150,000                                 | Maximum assets in the grid                  |
| `Na`             | 1000                                    | Number of asset grid points                 |
| `Nk`             | 11                                      | Number of labor supply grid points          |
| `l_grid`         | `range(0.0, 1.0, length=Nk)`            | Labor supply from 0 (no work) to 1 (full)   |
| `a_grid`         | `nonlinspace(0.0, a_max, Na, 5.0)`      | Nonlinear asset grid (curved toward 0)      |

In [ ]:
### Setup

In [2]:
using Random, Optim, LinearAlgebra, Interpolations
using Plots, Statistics, ProgressMeter, ForwardDiff

In [3]:



# -------------------------------
# Utility: Nonlinear Grid Creator
# -------------------------------
function nonlinspace(start::Float64, stop::Float64, num::Int, curv::Float64)
    lin_vals = range(0, stop=1, length=num)
    curved_vals = lin_vals .^ curv
    return start .+ (stop - start) .* curved_vals
end

@inline function util(model::DynLaborModel, c, h)
    # Utility from consumption and disutility from labor.
    # Note: we use par.phi (the weight on labor disutility) in place of a kids-adjusted parameter.
    par = model
    return (c^(1.0 - par.rho)) / (1.0 - par.rho) - par.phi * (h^(1.0 + par.eta)) / (1.0 + par.eta)
end

# -------------------------------
# Dynamic Labor Model Definition
# -------------------------------
mutable struct DynLaborModel
    T::Int                        # Time periods
    rho::Float64                 # Risk aversion (CRRA)
    beta::Float64                # Discount factor
    phi::Float64                 # Weight on labor disutility
    eta::Float64                 # Frisch elasticity parameter
    alpha::Float64               # (possibly productivity or returns to labor)
    w::Float64                   # Wage rate
    y::Float64                   # Non-labor income
    tau::Float64                 # Labor income tax
    r::Float64                   # Interest rate
    a_max::Float64               # Max asset level
    a_min::Float64               # Min asset level
    Na::Int                      # Number of asset grid points
    k_max::Float64               # Max labor effort
    Nk::Int                      # Number of labor grid points
    simT::Int                    # Simulation time periods
    simN::Int                    # Number of simulated agents
    a_grid::Vector{Float64}      # Asset grid
    k_grid::Vector{Float64}      # Labor grid
    sol_c::Array{Float64,3}      # Optimal consumption [T, Nn, Na, Nk]
    sol_h::Array{Float64,3}      # Optimal labor effort [T, Nn, Na, Nk]
    sol_v::Array{Float64,3}      # Value function [T, Nn, Na, Nk]
    sim_c::Array{Float64,2}      # Simulated consumption [simN, simT]
    sim_h::Array{Float64,2}      # Simulated labor [simN, simT]
    sim_a::Array{Float64,2}      # Simulated assets [simN, simT]
    sim_k::Array{Float64,2}      # Simulated labor choice [simN, simT]
    sim_a_init::Vector{Float64}  # Initial assets
    sim_k_init::Vector{Float64}  # Initial labor effort
    draws_uniform::Array{Float64,2}  # Uniform draws for simulation [simN, simT]
    w_vec::Vector{Float64}       # Time-varying wage vector [T]
end

# -------------------------------
# Constructor for DynLaborModel
# -------------------------------
function DynLaborModel(; T::Int=50, beta::Float64=0.98, rho::Float64=2.0, y::Float64=30_000.0,
                           r::Float64=0.02, a_max::Float64=300_000.0, Na::Int=100, simN::Int=5000,
                           a_min::Float64=0.0, k_max::Float64=50.0, Nk::Int=50,
                           w::Float64=20_000.0, tau::Float64=0.1,
                           eta::Float64=2.5, alpha::Float64=0.3,
                           phi::Float64=2.0, seed::Int=1234)

    # --- Time horizon and simulation settings ---
    simT = T

    # --- Grids for state variables and decisions ---
    a_grid = nonlinspace(a_min, a_max, Na, 2.0)
    k_grid = nonlinspace(0.0, k_max, Nk, 1.1)

    # --- Storage for solution (policy + value functions) ---
    # Dimensions: (T, Na, Nk) 
    sol_shape = (T, Na, Nk)
    sol_c = fill(NaN, sol_shape)   # Optimal consumption
    sol_h = fill(NaN, sol_shape)   # Optimal labor effort
    sol_v = fill(NaN, sol_shape)   # Value function

    # --- Simulation storage ---
    sim_shape = (simN, simT)
    sim_c = fill(NaN, sim_shape)
    sim_h = fill(NaN, sim_shape)
    sim_a = fill(NaN, sim_shape)
    sim_k = fill(NaN, sim_shape)

    # --- Random draws for simulation ---
    rng = MersenneTwister(seed)
    draws_uniform = rand(rng, sim_shape...)

    # --- Initial conditions for simulation ---
    sim_a_init = zeros(Float64, simN)
    sim_k_init = zeros(Float64, simN)

    # --- Wage vector (can vary by time) ---
    w_vec = fill(w, T)

    # --- Return constructed model ---
    return DynLaborModel(T, rho, beta, phi, eta, alpha, w, y, tau, r,
                          a_max, a_min, Na, k_max, Nk, simT, simN,
                          a_grid, k_grid,
                          sol_c, sol_h, sol_v,
                          sim_c, sim_h, sim_a, sim_k,
                          sim_a_init, sim_k_init, draws_uniform, w_vec)
end



# --------------------------
# Model Solver
# --------------------------

function solve_model!(model::DynLaborModel)
    T, Na, Nk = model.T, model.Na, model.Nk
    a_grid, k_grid = model.a_grid, model.k_grid
    sol_c, sol_h, sol_v = model.sol_c, model.sol_h, model.sol_v

    @showprogress 1 "Solving model..." for t in T:-1:1

        for (i_a, assets) in enumerate(a_grid)
            for (i_k, capital) in enumerate(k_grid)
                idx = (t, i_a, i_k)  # `1` is a dummy index to preserve 4D shape
                w = wage_func(model, capital, t)

                if t == T
                    # =============== LAST PERIOD (UNIVARIATE) ===============
                    f(hours) = obj_last(model, hours, assets, capital)

                    # Restrict h ∈ [0, 1]
                    lower_bound = 0.0
                    upper_bound = 1.0

                    res = optimize(f, lower_bound, upper_bound, GoldenSection())

                    h_opt = Optim.minimizer(res)
                    cons = cons_last(model, h_opt, capital, assets)

                    sol_h[idx...] = h_opt
                    sol_c[idx...] = cons
                    sol_v[idx...] = -Optim.minimum(res)

                else
                    # =============== EARLIER PERIODS (MULTIVARIATE) ===============

                    obj(x) = -value_of_choice(model, x[1], x[2], assets, capital, t)

                    # Initial guess from next period
                    idx_last = (t+1, i_a, i_k)
                    init = [sol_c[idx_last...], sol_h[idx_last...]]

                    # Box constraints: h ∈ [0, 1]
                    lb = [1e-6, 0.0]
                    ub = [Inf,   1.0]

                    res = optimize(obj, lb, ub, init,
                                   Fminbox(LBFGS()),
                                   Optim.Options(g_tol=1e-6))

                    sol_c[idx...] = Optim.minimizer(res)[1]
                    sol_h[idx...] = Optim.minimizer(res)[2]
                    sol_v[idx...] = -Optim.minimum(res)
                end
            end
        end
    end

    return model
end





@inline function cons_last(model::DynLaborModel, h::Float64, capital::Float64, assets::Float64)
    # Compute income from wages at the terminal period
    income = wage_func(model, capital, model.T) * h 
    # Terminal consumption: all available resources are consumed
    cons = assets + income
    return cons
end

@inline function obj_last(model::DynLaborModel, h::Float64, assets::Float64, capital::Float64)
    # Compute consumption in the last period given hours worked
    cons = cons_last(model, h, capital, assets)
    # Calculate the utility; note: using the modified util without fertility
    u = util(model, cons, h)
    # Objective for maximization (we minimize the negative utility)
    return -u
end



@inline function wage_func(model::DynLaborModel, capital::Float64, t::Int)
    # Compute the effective wage: after tax wage rate adjusted for capital effects.
    par = model  # Extract model parameters
    return (1.0 - par.tau) * par.w_vec[t] * (1.0 + par.alpha * capital)
end

function value_of_choice(model::DynLaborModel, cons, hours, assets, capital, t::Int)
    par = model
    sol_v = model.sol_v
    a_grid, k_grid = model.a_grid, model.k_grid

    # Apply penalties for constraint violations.
    penalty = 0.0
    penalty += (cons < 0.0)  ? cons * 10000.0 : 0.0
    penalty += (hours < 0.0) ? hours * 10000.0 : 0.0
    #penalty += (assets < 0.0) ? assets * 1000000.0 : 0.0

    # Current period utility.
    util_val = util(model, cons, hours)

    # Next period states:
    income = wage_func(model, capital, t) * hours
    a_next = (1.0 + par.r) * (assets + income - cons)
    k_next = capital + hours

    # Interpolate next period's value using 3D array indexing.
    interp = LinearInterpolation((a_grid, k_grid), sol_v[t+1, :, :], extrapolation_bc=Line())
    V_next = interp(a_next, k_next)

    return util_val + par.beta * V_next + penalty
end

@inline function neg_value_of_choice(x, model, assets, capital, t)
    # Return the negative total value for minimization.
    return -value_of_choice(model, x[1], x[2], assets, capital, t)
end

@inline function grad_neg_value_of_choice!(storage, x, model, assets, capital, t)
    # Use ForwardDiff to compute the gradient.
    ForwardDiff.gradient!(
        storage,
        y -> neg_value_of_choice(y, model, assets, capital, t),
        x
    )
end

function simulate_model!(model::DynLaborModel)
    simN, simT = model.simN, model.simT
    sim_a, sim_k = model.sim_a, model.sim_k
    sim_c, sim_h = model.sim_c, model.sim_h
    draws_uniform = model.draws_uniform

    # Precompute interpolation objects for each period.
    # sol_c and sol_h are assumed to be 3D arrays: dimensions (T, Na, Nk)
    interp_dict = Dict()
    for t in 1:simT
        sol_c_slice = model.sol_c[t, :, :]
        sol_h_slice = model.sol_h[t, :, :]
        interp_dict[(t, :c)] = LinearInterpolation((model.a_grid, model.k_grid), sol_c_slice, extrapolation_bc=Line())
        interp_dict[(t, :h)] = LinearInterpolation((model.a_grid, model.k_grid), sol_h_slice, extrapolation_bc=Line())
    end

    # Initialize simulation with initial asset and capital conditions.
    for i in 1:simN
        sim_a[i, 1] = model.sim_a_init[i]
        sim_k[i, 1] = model.sim_k_init[i]
    end

    # Simulation loop.
    for i in 1:simN
        for t in 1:simT
            sim_c[i, t] = interp_dict[(t, :c)](sim_a[i, t], sim_k[i, t])
            sim_h[i, t] = interp_dict[(t, :h)](sim_a[i, t], sim_k[i, t])
            # Ensure hours stay in [0, 1].
            sim_h[i, t] = clamp(sim_h[i, t], 0.0, 1.0)
            if t < simT
                income = wage_func(model, sim_k[i, t], t) * sim_h[i, t]
                sim_a[i, t+1] = (1 + model.r) * (sim_a[i, t] + income - sim_c[i, t])
                sim_k[i, t+1] = sim_k[i, t] + sim_h[i, t]
            end
        end
    end
end


LoadError: UndefVarError: `DynLaborModel` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [28]:
# --------------------------
# Main Routine
# --------------------------
model = DynLaborModel();
@time solve_model!(model);
simulate_model!(model);

Solving model...  22%|███████▊                           |  ETA: 0:01:18

LoadError: DomainError with -1.934949479944249e-6:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

In [2]:
using Logging, NLopt, ForwardDiff, Interpolations
using QuantEcon, FastGaussQuadrature, Random, Plots

In [3]:



# -------------------------------
# Utility: Nonlinear Grid Creator
# -------------------------------
function nonlinspace(start::Float64, stop::Float64, num::Int, curv::Float64)
    lin_vals = range(0, stop=1, length=num)
    curved_vals = lin_vals .^ curv
    return start .+ (stop - start) .* curved_vals
end

@inline function util(model::DynLaborModel, c, h)
    # Utility from consumption and disutility from labor.
    # Note: we use par.phi (the weight on labor disutility) in place of a kids-adjusted parameter.
    par = model
    return (c^(1.0 - par.rho)) / (1.0 - par.rho) - par.phi * (h^(1.0 + par.eta)) / (1.0 + par.eta)
end

# -------------------------------
# Dynamic Labor Model Definition
# -------------------------------
mutable struct DynLaborModel
    T::Int                        # Time periods
    rho::Float64                 # Risk aversion (CRRA)
    beta::Float64                # Discount factor
    phi::Float64                 # Weight on labor disutility
    eta::Float64                 # Frisch elasticity parameter
    alpha::Float64               # (possibly productivity or returns to labor)
    w::Float64                   # Wage rate
    y::Float64                   # Non-labor income
    tau::Float64                 # Labor income tax
    r::Float64                   # Interest rate
    a_max::Float64               # Max asset level
    a_min::Float64               # Min asset level
    Na::Int                      # Number of asset grid points
    k_max::Float64               # Max labor effort
    Nk::Int                      # Number of labor grid points
    simT::Int                    # Simulation time periods
    simN::Int                    # Number of simulated agents
    a_grid::Vector{Float64}      # Asset grid
    k_grid::Vector{Float64}      # Labor grid
    sol_c::Array{Float64,3}      # Optimal consumption [T, Nn, Na, Nk]
    sol_h::Array{Float64,3}      # Optimal labor effort [T, Nn, Na, Nk]
    sol_v::Array{Float64,3}      # Value function [T, Nn, Na, Nk]
    sim_c::Array{Float64,2}      # Simulated consumption [simN, simT]
    sim_h::Array{Float64,2}      # Simulated labor [simN, simT]
    sim_a::Array{Float64,2}      # Simulated assets [simN, simT]
    sim_k::Array{Float64,2}      # Simulated labor choice [simN, simT]
    sim_a_init::Vector{Float64}  # Initial assets
    sim_k_init::Vector{Float64}  # Initial labor effort
    draws_uniform::Array{Float64,2}  # Uniform draws for simulation [simN, simT]
    w_vec::Vector{Float64}       # Time-varying wage vector [T]
end

# -------------------------------
# Constructor for DynLaborModel
# -------------------------------
function DynLaborModel(; T::Int=50, beta::Float64=0.98, rho::Float64=2.0, y::Float64=30_000.0,
                           r::Float64=0.02, a_max::Float64=300_000.0, Na::Int=100, simN::Int=5000,
                           a_min::Float64=0.0, k_max::Float64=1.0, Nk::Int=50,
                           w::Float64=20_000.0, tau::Float64=0.1,
                           eta::Float64=2.5, alpha::Float64=0.3,
                           phi::Float64=2.0, seed::Int=1234)

    # --- Time horizon and simulation settings ---
    simT = T

    # --- Grids for state variables and decisions ---
    a_grid = nonlinspace(a_min, a_max, Na, 2.0)
    k_grid = nonlinspace(0.0, k_max, Nk, 1.1)

    # --- Storage for solution (policy + value functions) ---
    # Dimensions: (T, Na, Nk) 
    sol_shape = (T, Na, Nk)
    sol_c = fill(NaN, sol_shape)   # Optimal consumption
    sol_h = fill(NaN, sol_shape)   # Optimal labor effort
    sol_v = fill(NaN, sol_shape)   # Value function

    # --- Simulation storage ---
    sim_shape = (simN, simT)
    sim_c = fill(NaN, sim_shape)
    sim_h = fill(NaN, sim_shape)
    sim_a = fill(NaN, sim_shape)
    sim_k = fill(NaN, sim_shape)

    # --- Random draws for simulation ---
    rng = MersenneTwister(seed)
    draws_uniform = rand(rng, sim_shape...)

    # --- Initial conditions for simulation ---
    sim_a_init = zeros(Float64, simN)
    sim_k_init = zeros(Float64, simN)

    # --- Wage vector (can vary by time) ---
    w_vec = fill(w, T)

    # --- Return constructed model ---
    return DynLaborModel(T, rho, beta, phi, eta, alpha, w, y, tau, r,
                          a_max, a_min, Na, k_max, Nk, simT, simN,
                          a_grid, k_grid,
                          sol_c, sol_h, sol_v,
                          sim_c, sim_h, sim_a, sim_k,
                          sim_a_init, sim_k_init, draws_uniform, w_vec)
end



# --------------------------
# Model Solver
# --------------------------

function solve_model!(model::DynLaborModel)
    T, Na, Nk = model.T, model.Na, model.Nk
    a_grid, k_grid = model.a_grid, model.k_grid
    sol_c, sol_h, sol_v = model.sol_c, model.sol_h, model.sol_v

    @showprogress 1 "Solving model..." for t in T:-1:1

        for (i_a, assets) in enumerate(a_grid)
            for (i_k, capital) in enumerate(k_grid)
                idx = (t, i_a, i_k)  # `1` is a dummy index to preserve 4D shape
                w = wage_func(model, capital, t)

                if t == T
                    # =============== LAST PERIOD (UNIVARIATE) ===============
                    f(hours) = obj_last(model, hours, assets, capital)

                    # Restrict h ∈ [0, 1]
                    lower_bound = 0.0
                    upper_bound = 1.0

                    res = optimize(f, lower_bound, upper_bound, GoldenSection())

                    h_opt = Optim.minimizer(res)
                    cons = cons_last(model, h_opt, capital, assets)

                    sol_h[idx...] = h_opt
                    sol_c[idx...] = cons
                    sol_v[idx...] = -Optim.minimum(res)

                else
                    # =============== EARLIER PERIODS (MULTIVARIATE) ===============

                    obj(x) = -value_of_choice(model, x[1], x[2], assets, capital, t)

                    # Initial guess from next period
                    idx_last = (t+1, i_a, i_k)
                    init = [sol_c[idx_last...], sol_h[idx_last...]]

                    # Box constraints: h ∈ [0, 1]
                    lb = [1e-6, 0.0]
                    ub = [Inf,   1.0]

                    res = optimize(obj, lb, ub, init,
                                   Fminbox(LBFGS()),
                                   Optim.Options(g_tol=1e-6))

                    sol_c[idx...] = Optim.minimizer(res)[1]
                    sol_h[idx...] = Optim.minimizer(res)[2]
                    sol_v[idx...] = -Optim.minimum(res)
                end
            end
        end
    end

    return model
end
# ------------------------------------------------
# Supporting Functions (Deterministic version)
# ------------------------------------------------

@inline function cons_last(model::DynLaborModel, h::Float64, capital::Float64, assets::Float64)
    # In the last period, income is computed without shocks.
    income = wage_func(model, capital, model.T) * h
    cons = assets + income
    return cons
end

@inline function obj_last(model::DynLaborModel, h::Float64, assets::Float64, capital::Float64)
    cons = cons_last(model, h, capital, assets)
    u = util(model, cons, h)
    return -u
end



@inline function wage_func(model::DynLaborModel, capital::Float64, t::Int)
    par = model
    # Remove shocks: simply return the after-tax wage rate adjusted for capital effects.
    return (1.0 - par.tau) * par.w_vec[t] * (1.0 + par.alpha * capital)
end

function value_of_choice(model::DynLaborModel, cons, hours, assets, capital, t::Int)
    par = model
    a_grid, k_grid = model.a_grid, model.k_grid

    # Apply penalties if consumption or hours are negative.
    penalty = 0.0
    penalty += (cons < 0.0) ? cons * 10000.0 : 0.0
    penalty += (hours < 0.0) ? hours * 10000.0 : 0.0

    # Current period utility.
    util_val = util(model, cons, hours)

    # Next period states:
    income = wage_func(model, capital, t) * hours
    a_next = (1.0 + par.r) * (assets + income - cons)
    k_next = capital + hours

    # Interpolate next period's value function (3D array now: period x asset x capital).
    interp = LinearInterpolation((a_grid, k_grid), model.sol_v[t+1, :, :], extrapolation_bc=Line())
    V_next = interp(a_next, k_next)

    return util_val + par.rho * V_next + penalty
end

@inline function neg_value_of_choice(x, model, assets, capital, t)
    return -value_of_choice(model, x[1], x[2], assets, capital, t)
end

@inline function grad_neg_value_of_choice!(storage, x, model, assets, capital, t)
    ForwardDiff.gradient!(
        storage,
        y -> neg_value_of_choice(y, model, assets, capital, t),
        x
    )
end

# ------------------------------------------------
# NLopt Wrappers for Optimization (Deterministic version)
# ------------------------------------------------

# One-dimensional (scalar) optimization wrapper using NLopt.
function optimize_scalar(f, lb, ub, init)
    opt = Opt(:LN_BOBYQA, 1)
    lower_bounds!(opt, [lb])
    upper_bounds!(opt, [ub])
    xtol_rel!(opt, 1e-6)
    function obj_func(x, grad)
        return f(x[1])
    end
    min_objective!(opt, obj_func)
    (minf, minx, ret) = optimize(opt, [init])
    return minx[1], minf, ret
end

# Two-dimensional (vector) optimization wrapper using NLopt.
function optimize_vector(model, assets, capital, t, init)
    opt = Opt(:LD_LBFGS, 2)
    lower_bounds!(opt, [1e-6, 0.0])
    # No upper bound on consumption; hours bounded by [0, 24].
    upper_bounds!(opt, [Inf, 24.0])
    xtol_rel!(opt, 1e-6)
    function obj_func(x, grad)
         if length(grad) > 0
             grad_neg_value_of_choice!(grad, x, model, assets, capital, t)
         end
         return neg_value_of_choice(x, model, assets, capital, t)
    end
    min_objective!(opt, obj_func)
    (minf, minx, ret) = optimize(opt, init)
    return minx, minf, ret
end

# ------------------------------------------------
# Model Solver using NLopt (Deterministic version)
# ------------------------------------------------

function solve_model!(model::DynLaborModel)
    T, Na, Nk = model.T, model.Na, model.Nk
    a_grid, k_grid = model.a_grid, model.k_grid
    sol_c, sol_h, sol_v = model.sol_c, model.sol_h, model.sol_v

    for t in T:-1:1
        @info "Processing period t = $t"
        for (i_a, assets) in enumerate(a_grid)
            for (i_k, capital) in enumerate(k_grid)
                idx = (t, i_a, i_k)
                if t == T
                    # Last period: solve for optimal hours via scalar optimization.
                    hours_min = max((-assets / wage_func(model, capital, t)) + 1e-5, 0.0)
                    init_guess = (hours_min + 24.0) / 2
                    h_opt, fval, ret = optimize_scalar(x -> obj_last(model, x, assets, capital), hours_min, 24.0, init_guess)
                    cons = cons_last(model, h_opt, capital, assets)
                    sol_v[idx...] = util(model, cons, h_opt)
                    sol_h[idx...] = h_opt
                    sol_c[idx...] = cons
                else
                    # Earlier periods: solve jointly for consumption and hours.
                    init = [sol_c[t+1, i_a, i_k], sol_h[t+1, i_a, i_k]]
                    minx, minf, ret = optimize_vector(model, assets, capital, t, init)
                    sol_c[idx...] = minx[1]
                    sol_h[idx...] = minx[2]
                    sol_v[idx...] = -minf
                end
            end
        end
    end
    return model
end

# ------------------------------------------------
# Simulation (Deterministic version)
# ------------------------------------------------
  
function simulate_model!(model::DynLaborModel)
    simN, simT = model.simN, model.simT

    # Build interpolation objects for policy functions for each period.
    interp_dict = Dict{Tuple{Int,Symbol}, Any}()
    for t in 1:simT
        sol_c_slice = model.sol_c[t, :, :]
        sol_h_slice = model.sol_h[t, :, :]
        interp_dict[(t, :c)] = LinearInterpolation((model.a_grid, model.k_grid), sol_c_slice, extrapolation_bc=Line())
        interp_dict[(t, :h)] = LinearInterpolation((model.a_grid, model.k_grid), sol_h_slice, extrapolation_bc=Line())
    end

    # Initialize simulation state arrays.
    for i in 1:simN
        model.sim_a[i, 1] = model.sim_a_init[i]
        model.sim_k[i, 1] = model.sim_k_init[i]
    end

    # Simulation loop.
    for i in 1:simN
        for t in 1:simT
            a_current = model.sim_a[i, t]
            k_current = model.sim_k[i, t]
            model.sim_c[i, t] = interp_dict[(t, :c)](a_current, k_current)
            model.sim_h[i, t] = interp_dict[(t, :h)](a_current, k_current)
            income = wage_func(model, k_current, t) * model.sim_h[i, t]
            if t < simT
                model.sim_a[i, t+1] = (1 + model.r) * (a_current + income - model.sim_c[i, t])
                model.sim_k[i, t+1] = k_current + model.sim_h[i, t]
            end
        end
    end

    return model
end

LoadError: UndefVarError: `DynLaborModel` not defined in `Main`
Suggestion: check for spelling errors or missing imports.